In [21]:
import pandas as pd
import numpy as np
import pickle

In [22]:
df = pd.DataFrame()
with open('./JNLPBA/JNLPBA_protokol4.pkl', 'rb') as f:
    df = df.append(pickle.load(f))
with open('./CDR/CDR_protokol4.pkl', 'rb') as f:
    df = df.append(pickle.load(f))
with open('./CHEMDNER/CHEMDNER_protokol4.pkl', 'rb') as f:
    df = df.append(pickle.load(f))
with open('./BioRED/BioRED_protokol4.pkl', 'rb') as f:
    df = df.append(pickle.load(f))
with open('./NCBI/NCBI_protokol4.pkl', 'rb') as f:
    df = df.append(pickle.load(f))
with open('./N2C2/n2c2_protokol4.pkl', 'rb') as f:
    df = df.append(pickle.load(f))
df = df.reset_index(drop = True)

In [23]:
df

,ids,cor,class,text,labels
0,1,JNLPBA,[DNA],"[IL-2, gene, expression, and, NF-kappa, B, act...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,1,JNLPBA,[RNA],"[IL-2, gene, expression, and, NF-kappa, B, act...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,1,JNLPBA,"[Cell, Line]","[IL-2, gene, expression, and, NF-kappa, B, act...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,1,JNLPBA,"[Cell, Type]","[IL-2, gene, expression, and, NF-kappa, B, act...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,1,JNLPBA,[Protein],"[IL-2, gene, expression, and, NF-kappa, B, act...","[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]"
...,...,...,...,...,...
708785,324754,n2c2,[Form],"[Coronary, artery, disease, .]","[0, 0, 0, 0]"
708786,324755,n2c2,[Form],"[MAJOR, PROCEDURES, :, Intubation, .]","[0, 0, 0, 0, 0]"
708787,324756,n2c2,[Form],"[Arterial, line, .]","[0, 0, 0]"
708788,324757,n2c2,[Form],"[Central, line, .]","[0, 0, 0]"


In [24]:
df['labels_sum'] = df.labels.apply(lambda lst: sum(lst))
df['klasa'] = df["class"].apply(lambda lst: ' '.join(lst))
df_hasones = df[df.labels_sum!=0]
df_onlyzeros = df[df.labels_sum==0]

In [25]:
df_hasones_train = pd.DataFrame()
for i in df_hasones.klasa.unique():
    df_hasones_train = df_hasones_train.append(df_hasones[df_hasones.klasa == i].sample(frac = 0.85))

df_onlyzeros_train = pd.DataFrame()
for i in df_onlyzeros.klasa.unique():
    df_onlyzeros_train = df_onlyzeros_train.append(df_onlyzeros[df_onlyzeros.klasa == i].sample(frac = 0.85))

In [26]:
df_hasones_rest = df_hasones.drop(df_hasones_train.index)
df_onlyzeros_rest = df_onlyzeros.drop(df_onlyzeros_train.index)

In [27]:
df_hasones_valid = pd.DataFrame()
for i in df_hasones_rest.klasa.unique():
    df_hasones_valid = df_hasones_valid.append(df_hasones_rest[df_hasones_rest.klasa == i].sample(frac = 0.33))

df_onlyzeros_valid = pd.DataFrame()
for i in df_onlyzeros_rest.klasa.unique():
    df_onlyzeros_valid = df_onlyzeros_valid.append(df_onlyzeros_rest[df_onlyzeros_rest.klasa == i].sample(frac = 0.33))

In [28]:
df_hasones_test = df_hasones_rest.drop(df_hasones_valid.index)
df_onlyzeros_test = df_onlyzeros_rest.drop(df_onlyzeros_valid.index)

In [29]:
df_train = pd.concat([df_hasones_train, df_onlyzeros_train])

In [30]:
df_valid = pd.concat([df_hasones_valid, df_onlyzeros_valid])

In [31]:
df_test = pd.concat([df_hasones_test, df_onlyzeros_test])

In [32]:
print(df_onlyzeros_test.shape[0] + df_onlyzeros_valid.shape[0] + df_onlyzeros_train.shape[0] == df_onlyzeros.shape[0])
print(df_hasones_test.shape[0] + df_hasones_valid.shape[0] + df_hasones_train.shape[0] == df_hasones.shape[0])
print(df_test.shape[0] + df_valid.shape[0] + df_train.shape[0] == df.shape[0])
print(df_test.shape[0] / df.shape[0])
print(df_valid.shape[0] / df.shape[0])
print(df_train.shape[0] / df.shape[0])

True
True
True
0.10049944271222788
0.04950267357045105
0.8499978837173211


In [33]:
df_train.drop(['labels_sum'], inplace=True, axis=1)
df_train = df_train.reset_index(drop = True)
df_train

,ids,cor,class,text,labels,klasa
0,11708,JNLPBA,[DNA],"[Using, this, assay, ,, we, also, showed, that...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",DNA
1,20882,JNLPBA,[DNA],"[Our, study, demonstrated, that, all, samples,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",DNA
2,17362,JNLPBA,[DNA],"[Transcription, factor, IID, (, TFIID, ), bind...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...",DNA
3,2239,JNLPBA,[DNA],"[Structure, and, expression, of, the, human, G...","[0, 0, 0, 0, 0, 1, 1, 1, 0]",DNA
4,8722,JNLPBA,[DNA],"[Like, in, Th2, clones, ,, suppressed, NFAT-, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",DNA
...,...,...,...,...,...,...
602465,324444,n2c2,[Form],"[Denies, illicit, drug, use, Family, History, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Form
602466,310522,n2c2,[Form],"[Paraseptal, emphysema, is, mild, .]","[0, 0, 0, 0, 0]",Form
602467,299176,n2c2,[Form],"[[, *, *, Hospital, Unit, Name, 153, *, *, ], ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Form
602468,310775,n2c2,[Form],"[Unchanged, multiple, fat-containing, anterior...","[0, 0, 0, 0, 0, 0, 0, 0]",Form


In [34]:
df_valid.drop(['labels_sum'], inplace=True, axis=1)
df_valid = df_valid.reset_index(drop = True)
df_valid

,ids,cor,class,text,labels,klasa
0,16505,JNLPBA,[DNA],"[Taken, together, ,, the, results, indicate, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",DNA
1,17813,JNLPBA,[DNA],"[Ectopic, overexpression, of, IRF-1, results, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...",DNA
2,20490,JNLPBA,[DNA],"[Mutation, of, the, AP-1, site, not, only, abo...","[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",DNA
3,10960,JNLPBA,[DNA],"[Transcription, factor, GATA-3, is, differenti...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",DNA
4,13504,JNLPBA,[DNA],"[Altered, DNA-binding, specificity, mutants, o...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",DNA
...,...,...,...,...,...,...
35082,315043,n2c2,[Form],"[We, have, made, no, changes, to, your, medica...","[0, 0, 0, 0, 0, 0, 0, 0, 0]",Form
35083,321215,n2c2,[Form],"[Small, amount, of, pelvic, free, fluid, ,, fl...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Form
35084,323956,n2c2,[Form],"[She, had, episodes, of, non-responsiveness, o...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Form
35085,315645,n2c2,[Form],"[-, CAD, s/p, stent, [, *, *, 2-18, *, *, ], (...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Form


In [35]:
df_test.drop(['labels_sum'], inplace=True, axis=1)
df_test = df_test.reset_index(drop = True)
df_test

,ids,cor,class,text,labels,klasa
0,7,JNLPBA,[DNA],"[The, peri-kappa, B, site, mediates, human, im...","[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...",DNA
1,13,JNLPBA,"[Cell, Line]","[This, site, is, conserved, among, isolates, o...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Cell Line
2,17,JNLPBA,[Protein],"[Understanding, the, monocyte-specific, functi...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...",Protein
3,18,JNLPBA,[DNA],"[E1A, gene, expression, induces, susceptibilit...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",DNA
4,24,JNLPBA,"[Cell, Type]","[The, CD4, coreceptor, interacts, with, non-po...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Cell Type
...,...,...,...,...,...,...
71228,324717,n2c2,[Form],"[Hypotension, -, The, patient, was, initially,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Form
71229,324726,n2c2,[Form],"[He, was, continued, on, his, Aspirin, and, hi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Form
71230,324733,n2c2,[Form],"[Code, status, -, The, patient, confirmed, tha...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Form
71231,324742,n2c2,[Form],"[DISCHARGE, MEDICATIONS, :, 1, .]","[0, 0, 0, 0, 0]",Form


In [36]:
# using protocol 4 for dumping a .pkl file

filename = './ALL_DATA_klasa_TRAIN_protokol4.pkl'
outfile = open(filename,'wb')
pickle.dump(df_train,outfile, protocol=4)

In [37]:
# using protocol 4 for dumping a .pkl file

filename = './ALL_DATA_klasa_VALID_protokol4.pkl'
outfile = open(filename,'wb')
pickle.dump(df_valid,outfile, protocol=4)

In [38]:
# using protocol 4 for dumping a .pkl file

filename = './ALL_DATA_klasa_TEST_protokol4.pkl'
outfile = open(filename,'wb')
pickle.dump(df_test,outfile, protocol=4)